# LangChain Quickstart
This quickstart is meant to guide the user through the basics of building and interacting with a LLM using langchain
It is update from March 2024, and based of the LangChain docs found here - https://python.langchain.com/docs/get_started/quickstart

In [44]:
%pip install langchain langchain-openai

Note: you may need to restart the kernel to use updated packages.


In [45]:
# # Let's make this work with Google Colab by default. Uncomments the commented items below if you want to use this on a local notebook
# import getpass
# import os
# import uuid
# from google.colab import userdata

# os.environ["OPENAI_API_KEY"] = userdata.get('OPENAI_API_KEY')
# os.environ["LANGCHAIN_API_KEY"] = userdata.get('LANGCHAIN_API_KEY')

# def _set_if_undefined(var: str):
#     if not os.environ.get(var):
#         os.environ[var] = getpass(f"Please provide your {var}")


# _set_if_undefined("OPENAI_API_KEY")
# _set_if_undefined("LANGCHAIN_API_KEY")

# Optional, add tracing in LangSmith.
# This will help you visualize and debug the control flow
# os.environ["LANGCHAIN_TRACING_V2"] = "true"
# os.environ["LANGCHAIN_PROJECT"] = "quickstart"

# Uncomment below if you want to enter the API key manually
#os.environ["OPENAI_API_KEY"] = getpass.getpass()

#Uncomment below if you want to use .env file
import os
import uuid
import dotenv

#load the .env file from the default location
dotenv.load_dotenv()

#retrieve the openai key
openai_api_key = os.getenv("OPENAI_API_KEY")

#check if OPENAI_API_KEY is not NONE and set it
if openai_api_key:
    os.environ["OPENAI_API_KEY"] = openai_api_key
else:
    # If OPENA_AI_API_KEY is not set, prompt the user to enter the key
    print("OPENAI_API_KEY not found in the .env file")
# Optional, add tracing in LangSmith.
os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_PROJECT"] = "quickstart"

In [46]:
from langchain_openai import ChatOpenAI
llm = ChatOpenAI()

In [47]:
llm.invoke("What great bands are playing music in Austin tonight?")

AIMessage(content="I'm not able to provide real-time information, but some great bands that frequently play in Austin include Spoon, White Denim, Black Pumas, Shakey Graves, and Grupo Fantasma. You can check local music venues and event listings to see who might be playing tonight.", response_metadata={'token_usage': {'completion_tokens': 56, 'prompt_tokens': 17, 'total_tokens': 73}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': 'fp_3bc1b5746c', 'finish_reason': 'stop', 'logprobs': None})

In [48]:
from langchain_core.prompts import ChatPromptTemplate
prompt = ChatPromptTemplate.from_messages([
    ("system", "You are world class event planner, with amazing documentation skills."),
    ("user", "{input}")
])

In [49]:
chain = prompt | llm

In [50]:
chain.invoke({"input": "What great bands are playing music in Austin tonight?"})

AIMessage(content='I can help you find some great bands playing in Austin tonight! Do you have a specific genre or venue in mind, or are you open to any suggestions?', response_metadata={'token_usage': {'completion_tokens': 32, 'prompt_tokens': 33, 'total_tokens': 65}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': 'fp_3bc1b5746c', 'finish_reason': 'stop', 'logprobs': None})

The default output of ChatModel is a message. It's more convienient to work with strings. We'll use StrOutputParser to convert the chat message to a string.

In [51]:
from langchain_core.output_parsers import StrOutputParser
output_parser = StrOutputParser()

We cha now add this parser to the previous chain

In [52]:
chain = prompt | llm | output_parser

we can now invoke it and ask the same question

In [53]:
chain.invoke({"input": "What great bands are playing music in Austin tonight?"})

"Tonight in Austin, you have a variety of fantastic bands to choose from! Here are some of the top picks for live music:\n\n1. The Black Angels at Mohawk Austin\n2. Shakey Graves at Stubbs BBQ\n3. White Denim at Antone's Nightclub\n4. Gary Clark Jr. at ACL Live at the Moody Theater\n5. Spoon at The Continental Club\n6. Bright Light Social Hour at Barracuda Austin\n\nThese are just a few options, and there are many more talented bands and artists playing in Austin tonight. Enjoy the live music scene in the Live Music Capital of the World!"

## Retrieval Chain

In order to properly answer our question, we are going to need to retrieve information from the web. We can grab information from the internet using the WebBaseLoader. This loader will allow us to retrieve information from the web and use it in our model.

In [54]:
#install beautifulsoup4
%pip install beautifulsoup4

Note: you may need to restart the kernel to use updated packages.


Lets pull the events down from the Austin Chronicle using WebBaseLoader from langchain_community

In [55]:
from langchain_community.document_loaders import WebBaseLoader
loader = WebBaseLoader("https://www.austinchronicle.com/events/")

docs = loader.load()

## Vector Store
Now we need to take the data we pulled from the web and convert it into a format that the model can understand. 


In [56]:
from langchain_openai import OpenAIEmbeddings
embeddings = OpenAIEmbeddings()

Let's use faiss, a very simple in memory vector store, to store the vectors of the events we pulled from the web.

In [57]:
%pip install faiss-cpu

Note: you may need to restart the kernel to use updated packages.


In [58]:
# Now we can build our index

from langchain_community.vectorstores import FAISS
from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter()
documents = text_splitter.split_documents(docs)
vector = FAISS.from_documents(documents, embeddings)

Now that we have taken our data, split it up and stuffed into in a vector store, we can build our retrieval chain. 

In [59]:
from langchain.chains.combine_documents import create_stuff_documents_chain

prompt = ChatPromptTemplate.from_template("""Answer the following question based only on the provided context:

<context>
{context}
</context>
                                        
Question: {input}""")

document_chain = create_stuff_documents_chain(llm, prompt)

If we wanted to simplify things, we can pass this in directly to the retrieval chain.


In [60]:
from langchain_core.documents import Document

document_chain.invoke({
    "input": "What great bands are playing music in Austin tonight?",
    "context": [Document(page_content="A great honkey tonk band is always playing at the White Horse")]
})

'Based on the context provided, a great honkey tonk band is always playing at the White Horse in Austin.'

However, we want to pull this from our vector store, which was populated by the WebBaseLoder

In [61]:
from langchain.chains import create_retrieval_chain

retriever = vector.as_retriever()
retrieval_chain = create_retrieval_chain(retriever, document_chain)

We can now invoke our chain. This returns a dictionary, with the response from the LLM in the answer key

In [62]:
response = retrieval_chain.invoke({"input": "What fun bands are playing in Austin tonight?"})
print(response["answer"])

Chris Gage, David Touchton & Shad Blair, Erick the Architect, Eve Monsees & Mike Buck, Flyjack, Herb Alpert & Lani Hall, Jessica Healey, Lonelyland, Pat Byrne Band, Matt Hansen, The 4411, zachy, Michael Hale Trio, and Motown Monday w/ Matchmaker Band are among the fun bands playing in Austin tonight.


#Conversation Retriever Chain
What we have done so far is useful, but we can make it more useful by adding a conversation retriever chain. This chain will allow us to pull in previous conversations and use them to help answer the question.

We can still use create_retrieval_chain function, but we need to change a couple things.
1. The retrieval method should work on our entire conversation history, not just the most recent input
2. The final LLM chain should likewise take the entire conversation history as input


In [63]:
from langchain.chains import create_history_aware_retriever
from langchain_core.prompts import MessagesPlaceholder

# First, we need a prompt that we can pass into a LLM to generate this search query

prompt = ChatPromptTemplate.from_messages([
    MessagesPlaceholder(variable_name="chat_history"),
    ("user"), "{input}",
    ("user"), "Given the above converation, generate a search query to lookup in order to get information relevant to the conversation"
])

retriever_chain = create_history_aware_retriever(llm, retriever, prompt)

We can test this by asking a follow up question 

In [64]:
from langchain_core.messages import HumanMessage, AIMessage

chat_history = [HumanMessage(content="Can I go to Motown Monday??"), AIMessage(content="Yes!")]
retriever_chain.invoke({
    "chat_history": chat_history,
    "input": "Tell me where Motown Monday is"
})


[Document(page_content="Music\n\n\n\n\nChris Gage\nMon., March 25, 9pm\nDonn's Depot\n\n\n\n\nMusic\n\n\n\n\nChurch on Monday w/ Elias Haslanger & Dr. James Polk\nMondays, 8:30pm and Mondays, 8:30pm\nContinental Club Gallery\n\n\n\n\nMusic\n\n\n\n\nConcerts in the Dark w/ Will Taylor & Tony Rogers\nMon., March 25, 7:30pm \xa0 get tickets\nATX Unplugged\n\n\n\n\nMusic\n\n\n\n\nDavid Touchton & Shad Blair song swap\nEvery fourth Monday, 8:30pm\nGiddy Ups\n\n\n\n\nMusic\n\n\n\n\nErick the Architect, 1010benja\nMon., March 25, 8pm \xa0 get tickets\nCome & Take It Live\n\n\n\n\nMusic\n\n\n\n\nEve Monsees & Mike Buck\nMon., March 25, 6:30pm. No cover (21+).\nContinental Club\n\n\n\n\nMusic\n\n\n\n\nFlyjack\nMon., March 25, 9:30pm. $7 cover (21+).\nC-Boy's Heart & Soul\n\n\n\n\nCommunity\n\n\nEvents\n\n\n\n\n\n\n\n\n\nFree Monday Tango\n\n              A delightful night of dance starts with an intro to authentic Argentine tango (no experience or partner needed) with Gustavo Simplis, and cont

What you should see is a list of documents returned that may have information similar to the question asked. This is because the retriever chain is going through the entire conversation history

In [65]:
prompt = ChatPromptTemplate.from_messages([
    ("system", "Answer the questions based on teh below context:\n\n{context}"),
    MessagesPlaceholder(variable_name="chat_history"),
    ("user", "{input}"),
])

document_chain = create_stuff_documents_chain(llm, prompt)

retrieval_chain = create_retrieval_chain(retriever_chain, document_chain)

Now we can test this end to end by asking a follow up question

In [80]:
from langchain.tools.retriever  import create_retriever_tool

retriever_tool = create_retriever_tool(
    retriever,
    "Austin_Events_Search",
    "Search for information about events in Austin. For any questions about austin events, you may use this tool",
)

Now let's define a tool to search. In the langchain documentation, the example uses a search tool called Tavily. 
This tool has a pretty generous free tier that should be more than enough for our purposes.



let's install duckduckgosearch using pip

In [81]:
%pip install duckduckgo-search

Note: you may need to restart the kernel to use updated packages.


In [1]:
from langchain_community.tools import DuckDuckGoSearchRun

search = DuckDuckGoSearchRun()

In [83]:
tools = [retriever_tool, search]

Now that we have our tools defined, we can create an agent to use them. This will be a very simple instance of an agent. 

In [84]:
pip install langchainhub

Note: you may need to restart the kernel to use updated packages.


Let's use langchainhub to pull down a pre-defined prompt

In [85]:
from langchain_openai import ChatOpenAI
from langchain import hub
from langchain.agents import create_openai_functions_agent
from langchain.agents import AgentExecutor

# Get the prompt to use - (you can pull whatever prompt you want here, feel free to modify)
prompt = hub.pull("hwchase17/openai-functions-agent")
llm = ChatOpenAI(model="gpt-3.5-turbo", temperature=0)
agent = create_openai_functions_agent(llm, tools, prompt)
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)


Now we can invoke our agent, and start asking it questions about austin nightlife  

In [86]:
agent_executor.invoke({"input": "What great bands are playing music in Austin tonight?"})



> Entering new AgentExecutor chain...

Invoking: `Austin_Events_Search` with `{'query': 'live music bands playing in Austin tonight'}`


Music




Chris Gage
Mon., March 25, 9pm
Donn's Depot




Music




Church on Monday w/ Elias Haslanger & Dr. James Polk
Mondays, 8:30pm and Mondays, 8:30pm
Continental Club Gallery




Music




Concerts in the Dark w/ Will Taylor & Tony Rogers
Mon., March 25, 7:30pm   get tickets
ATX Unplugged




Music




David Touchton & Shad Blair song swap
Every fourth Monday, 8:30pm
Giddy Ups




Music




Erick the Architect, 1010benja
Mon., March 25, 8pm   get tickets
Come & Take It Live




Music




Eve Monsees & Mike Buck
Mon., March 25, 6:30pm. No cover (21+).
Continental Club




Music




Flyjack
Mon., March 25, 9:30pm. $7 cover (21+).
C-Boy's Heart & Soul




Community


Events









Free Monday Tango

              A delightful night of dance starts with an intro to authentic Argentine tango (no experience or partner needed) with Gustavo Simplis

{'input': 'What great bands are playing music in Austin tonight?',
 'output': "Here are some great bands playing music in Austin tonight:\n\n1. Chris Gage at Donn's Depot at 9pm\n2. Church on Monday with Elias Haslanger & Dr. James Polk at Continental Club Gallery at 8:30pm\n3. Concerts in the Dark with Will Taylor & Tony Rogers at ATX Unplugged at 7:30pm\n4. David Touchton & Shad Blair song swap at Giddy Ups at 8:30pm\n5. Erick the Architect, 1010benja at Come & Take It Live at 8pm\n\nThese are just a few of the bands performing tonight in Austin. Enjoy the music scene!"}

Let's use this tool to ask about meetups in Austin tonight

In [89]:
agent_executor.invoke({"input": "What unique and fun events are going on tonight in Austin?"})



> Entering new AgentExecutor chain...

Invoking: `Austin_Events_Search` with `{'query': 'unique and fun events in Austin tonight'}`


Events - Monday, March 25, 2024 - The Austin Chronicle






























 CATEGORIES
 SEARCH







 ADVERTISE




 SEARCH




      All Events
    
      Music
    
      Movies
    
      Food
    
      Community
    
      Arts
    
      Queer
    
      Blog
    
      Contests
    
      Chronicle
    
      Support Us
    





CANCEL




Events Home

The Austin Chronicle

Austin FC
Elections
Op-Ed Submissions
Online Store
Summer Camps

Live Music

Roadshows
Live Music Venues

Movies: New This Week

Showtimes by Theatre
Showtimes by Movie
Special Screenings
Movie Review Archive

Food & Drink


Community Events

General Events
Kids
Out of Town
Sports & Fitness
Civic Events
Seasonal Jobs

Arts Events

Visual Arts
Theatre
Comedy
Books
Dance
Classical Music

Qmmunity Events

Nightlife & Parties
Arts & Culture
Community

Chron Events Blo

{'input': 'What unique and fun events are going on tonight in Austin?',
 'output': "Here are some unique and fun events happening tonight in Austin:\n\n1. Free Monday Tango at Halcyon Mueller - A night of dance starting with an intro to authentic Argentine tango and followed by social dancing.\n2. Grackle Games at Oilcan's - A competition drag show featuring talented drag performers going head to head in creative challenges.\n3. Nude Model Sketching at Atelier Dojo - A session for capturing unclothed human figures on canvas or paper with pigments.\n4. Horror Movie Trivia at Captain Quackenbush's - Test your knowledge of horror movies with trivia and win fun prizes.\n\nThese are just a few of the events happening tonight in Austin. Enjoy your evening!"}

In [90]:
agent_executor.invoke({"input": "What are some fun things to do in San Francisco tonight?"})



> Entering new AgentExecutor chain...

Invoking: `duckduckgo_search` with `{'query': 'fun things to do in San Francisco tonight'}`


Today's Most Popular Events. Penumbral Lunar Eclipse Night Over San Francisco. 2024 Sucka Flea Market & Swap (SF) "Dye Hard" Easter Egg Hunt + Festival at Kapwa Gardens (SF) 2024 Oakland Restaurant Week Final Day (March 14-24) Free Reggae in the Park 2024 "Crucial Sundays" (Golden Gate Park) < Saturday, March 23 Monday, March 25 >. SF's HellaDesi Comedy Night (Every Saturday) Saturday, March 23 - 7:00 pm | Cost: FREE* | Neck of The Woods. It's San Francisco's only weekly HellaDesi Comedy Night at a great new comedy club at Neck of the Woods with HellaFunny and Funcheap. *Special Sept. 30, 2023 Special Location SAVOY TIVOLI in North Beach! March 22-24. Sonoma, $75 and up. Cheese, please! Spend your entire weekend indulging in all kinds of artisan cheeses at this annual festival, which features farm and producer tours, seminars ... Events in Bay Area. SF 

{'input': 'What are some fun things to do in San Francisco tonight?',
 'output': 'Here are some fun things to do in San Francisco tonight:\n\n1. Penumbral Lunar Eclipse Night Over San Francisco\n2. 2024 Sucka Flea Market & Swap\n3. "Dye Hard" Easter Egg Hunt + Festival at Kapwa Gardens\n4. 2024 Oakland Restaurant Week Final Day\n5. Free Reggae in the Park\n6. "Crucial Sundays" at Golden Gate Park\n7. SF\'s HellaDesi Comedy Night at Neck of The Woods\n\nYou can also consider visiting Sonoma for a cheese festival or exploring the city with an audio tour or riding a cable car. Enjoy your evening in San Francisco!'}

Now let's have some conversations with agent

In [91]:
chat_history = [HumanMessage(content="Can I find fun things to do in Austin tonight?"), AIMessage(content="Yes!")]
agent_executor.invoke({
    "chat_history": chat_history,
    "input": "Tell me where Motown Monday is"
})



> Entering new AgentExecutor chain...

Invoking: `Austin_Events_Search` with `{'query': 'Motown Monday Austin'}`


Music




Chris Gage
Mon., March 25, 9pm
Donn's Depot




Music




Church on Monday w/ Elias Haslanger & Dr. James Polk
Mondays, 8:30pm and Mondays, 8:30pm
Continental Club Gallery




Music




Concerts in the Dark w/ Will Taylor & Tony Rogers
Mon., March 25, 7:30pm   get tickets
ATX Unplugged




Music




David Touchton & Shad Blair song swap
Every fourth Monday, 8:30pm
Giddy Ups




Music




Erick the Architect, 1010benja
Mon., March 25, 8pm   get tickets
Come & Take It Live




Music




Eve Monsees & Mike Buck
Mon., March 25, 6:30pm. No cover (21+).
Continental Club




Music




Flyjack
Mon., March 25, 9:30pm. $7 cover (21+).
C-Boy's Heart & Soul




Community


Events









Free Monday Tango

              A delightful night of dance starts with an intro to authentic Argentine tango (no experience or partner needed) with Gustavo Simplis, and continues with t

{'chat_history': [HumanMessage(content='Can I find fun things to do in Austin tonight?'),
  AIMessage(content='Yes!')],
 'input': 'Tell me where Motown Monday is',
 'output': 'Motown Monday is at The Highball in Austin tonight at 8:30pm. You can get tickets for the event. Enjoy the music and have a great time!'}

## Did we use the retrieval chain or just search?


Our agent is configured to use both the retrieval chain we've built from the web-loaded documents (i.e., events information from the Austin Chronicle) and a search tool (DuckDuckGoSearchRun). When we invoke the `agent_executor` with a question, the agent decides dynamically which tools to use to answer the question. The decision-making process is influenced by the prompt we're using (`"hwchase17/openai-functions-agent"`) and how the agent is designed to interpret the input and choose the appropriate tool(s).

prompt:

```
SYSTEM
You are a helpful assistant

PLACEHOLDER
chat_history

HUMAN
{input}

PLACEHOLDER
agent_scratchpad
``` 

Here's a breakdown of the components and how they might be used by our agent:

1. **Retriever Tool (`retriever_tool`)**: This tool is designed to search within the information we've indexed from the Austin Chronicle. It uses our retrieval chain to find relevant events based on the user's input.

2. **Search Tool (`DuckDuckGoSearchRun`)**: This tool performs a broader web search using DuckDuckGo. It's useful for questions that cannot be answered solely based on the indexed events from the Austin Chronicle.

When we ask questions like "What great bands are playing music in Austin tonight?" or "What unique and fun events are going on tonight in Austin?", the agent decides in real-time whether to pull information from the indexed events (using the retriever tool) or to perform a web search (using the DuckDuckGo search tool). The decision is influenced by several factors, including:

- The nature of the question: Whether it's more likely to be answered by the indexed events or requires broader web search.
- The capabilities and configurations of the agent: How the `create_openai_functions_agent` and its prompt are designed to interpret the input and select the appropriate tool.

To determine whether the agent is pulling from the retrieval chain or just using search for a given question, we'd need to inspect the agent's behavior or its output. If the agent is configured to log or otherwise indicate which tool it's using for a given response, that information can tell us whether it's relying on the retrieval chain, the search tool, or both. The verbose output from `AgentExecutor` (`verbose=True`) might provide insights into which tool(s) the agent decides to use for answering each question.